In [7]:
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance
from dotenv import load_dotenv
import os

load_dotenv()  # Load environment variables from .env file

cloud_client = QdrantClient(
    url=os.getenv("QDRANT_URL"),
    api_key=os.getenv("QDRANT_API_KEY")
)

if not cloud_client.collection_exists("resumes"):
    cloud_client.create_collection(
        collection_name="resumes",
        vectors_config=VectorParams(
            size=384,   # SAME AS LOCAL
            distance=Distance.COSINE
        )
    )
    print("✅ Cloud Collection created")
else:
    print("ℹ️ Cloud Collection already exists")
info = cloud_client.get_collection("resumes")
print(info)

ℹ️ Cloud Collection already exists
status=<CollectionStatus.GREEN: 'green'> optimizer_status=<OptimizersStatusOneOf.OK: 'ok'> warnings=None indexed_vectors_count=0 points_count=243 segments_count=2 config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=384, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None, datatype=None, multivector_config=None), shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None, inline_storage=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=10000, flush_interval_sec=5, max_optimization_threads=None), wal_config=WalConfig(wal_capacity_mb=

In [6]:
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct
from dotenv import load_dotenv

import os 

load_dotenv()  # Load environment variables from .env file

# Local Docker
local_client = QdrantClient(url="http://localhost:6333")

# Cloud
cloud_client = QdrantClient(
    url=os.getenv("QDRANT_URL"),
    api_key=os.getenv("QDRANT_API_KEY"),
    timeout=120
)

collection_name = "resumes"
batch_size = 50
offset = None
total = 0

print("🚀 Starting migration...")

while True:
    records, offset = local_client.scroll(
        collection_name=collection_name,
        limit=batch_size,
        offset=offset,
        with_vectors=True,
        with_payload=True
    )

    if not records:
        break

    points = [
        PointStruct(
            id=record.id,
            vector=record.vector,
            payload=record.payload
        )
        for record in records
    ]

    cloud_client.upsert(
        collection_name=collection_name,
        points=points
    )

    total += len(points)
    print(f"Transferred: {total}")

    if offset is None:
        break

print("✅ Migration Complete")


🚀 Starting migration...
Transferred: 50
Transferred: 100
Transferred: 150
Transferred: 200
Transferred: 243
✅ Migration Complete


In [10]:
import io
import os
import time
import uuid
import boto3
import fitz  # PyMuPDF
from dotenv import load_dotenv
from pymongo import MongoClient
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct, VectorParams, Distance
from sentence_transformers import SentenceTransformer
from PIL import Image
from pdf2image import convert_from_bytes
import pytesseract

# ================= TESSERACT CONFIG =================
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files (x86)\Tesseract-OCR\tesseract.exe"
os.environ["TESSDATA_PREFIX"] = r"C:\Program Files (x86)\Tesseract-OCR"

# ================= LOAD ENV =================
load_dotenv()

AWS_ACCESS_KEY = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")
AWS_REGION = os.getenv("AWS_REGION")
S3_BUCKET = os.getenv("AWS_S3_BUCKET")

MONGO_URI = os.getenv("MONGODB_URI")
QDRANT_URL = os.getenv("QDRANT_URL")
QDRANT_API_KEY = os.getenv("QDRANT_API_KEY")  # <--- new

# ================= CHECK ENV =================
for var_name, var in [
    ("AWS_ACCESS_KEY", AWS_ACCESS_KEY),
    ("AWS_SECRET_KEY", AWS_SECRET_KEY),
    ("AWS_REGION", AWS_REGION),
    ("S3_BUCKET", S3_BUCKET),
    ("MONGO_URI", MONGO_URI),
    ("QDRANT_URL", QDRANT_URL),
    ("QDRANT_API_KEY", QDRANT_API_KEY)
]:
    if not var:
        raise RuntimeError(f"❌ Environment variable {var_name} missing")

DB_NAME = "ats"
COLLECTION_NAME = "applications"
QDRANT_COLLECTION = "resumes"
BATCH_SIZE = 10
MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"

# ================= START =================
print("\n🤖 AGENT STARTED: RESUME INDEXING")
print("📊 MongoDB + AWS S3 + Qdrant")

# ================= AWS =================
s3 = boto3.client(
    "s3",
    aws_access_key_id=AWS_ACCESS_KEY,
    aws_secret_access_key=AWS_SECRET_KEY,
    region_name=AWS_REGION
)
s3.list_buckets()
print("✅ AWS verified")

# ================= MongoDB =================
mongo = MongoClient(MONGO_URI)
db = mongo[DB_NAME]
applications = db[COLLECTION_NAME]
print("✅ MongoDB connected")

# ================= QDRANT =================
qdrant = QdrantClient(
    url=QDRANT_URL,
    api_key=QDRANT_API_KEY,
    prefer_grpc=False,  # REST API (required for Cloud)
    timeout=120
)

# Ensure collection exists
if not qdrant.collection_exists(QDRANT_COLLECTION):
    qdrant.create_collection(
        collection_name=QDRANT_COLLECTION,
        vectors_config=VectorParams(
            size=384,
            distance=Distance.COSINE
        )
    )
    print("✅ Qdrant collection created")
else:
    print("ℹ️ Qdrant collection exists")

# ================= MODEL =================
model = SentenceTransformer(MODEL_NAME)
print("✅ Embedding model loaded")

# ================= HELPERS =================
def mongo_id_to_uuid(mongo_id: str) -> str:
    return str(uuid.uuid5(uuid.NAMESPACE_DNS, mongo_id))

def extract_s3_key(url: str) -> str:
    return url.split(".amazonaws.com/")[-1]

def extract_text_from_s3(url: str) -> str:
    key = extract_s3_key(url)
    print(f"   📥 Downloading: {key}")

    obj = s3.get_object(Bucket=S3_BUCKET, Key=key)
    file_bytes = obj["Body"].read()
    text = ""

    # ---------- IMAGE ----------
    if key.lower().endswith((".jpg", ".jpeg", ".png")):
        image = Image.open(io.BytesIO(file_bytes))
        text = pytesseract.image_to_string(image)

    # ---------- PDF ----------
    else:
        with fitz.open(stream=file_bytes, filetype="pdf") as doc:
            for page in doc:
                text += page.get_text()
        # OCR fallback if PDF has no extractable text
        if not text.strip():
            images = convert_from_bytes(file_bytes)
            for img in images:
                text += pytesseract.image_to_string(img)

    if not text.strip():
        raise ValueError("No text extracted")

    print(f"   📄 Extracted {len(text)} chars")
    return text


# ================= AGENT =================
def resume_indexing_agent():
    """
    Only index resumes whose resume_status is 'open'.
    """
    while True:
        # ----------------- QUERY -----------------
        query = {
            "resume": {"$exists": True, "$regex": "^http"},
            "resume_status": "open",  # ONLY open resumes
            "$or": [
                {"rag_uploaded": False},
                {"rag_uploaded": "False"},  # in case it is stored as string
                {"rag_uploaded": {"$exists": False}}
            ]
        }

        pending_count = applications.count_documents(query)
        print(f"📌 Pending/open resumes count: {pending_count}")

        batch = list(applications.find(query).limit(BATCH_SIZE))
        if not batch:
            print("✅ No open resumes pending")
            break

        points = []

        for app in batch:
            app_id = str(app["_id"])
            job_id = str(app.get("jobID", ""))
            print(f"\n📄 Processing {app_id} | Job: {job_id}")

            try:
                text = extract_text_from_s3(app["resume"])
                embedding = model.encode(text).tolist()

                points.append(
                    PointStruct(
                        id=mongo_id_to_uuid(app_id),
                        vector=embedding,
                        payload={
                            "application_id": app_id,
                            "job_id": job_id,
                            "resume_text": text[:1500]
                        }
                    )
                )

                # Update only open resumes
                applications.update_one(
                    {"_id": app["_id"]},
                    {"$set": {
                        "resume_status": "indexed",
                        "rag_uploaded": True,
                        "indexed_at": time.time()
                    }}
                )
                print("✅ Indexed")

            except Exception as e:
                applications.update_one(
                    {"_id": app["_id"]},
                    {"$set": {
                        "resume_status": "failed",
                        "error": str(e)
                    }}
                )
                print(f"❌ Failed → {e}")

        if points:
            qdrant.upsert(
                collection_name=QDRANT_COLLECTION,
                points=points
            )
            print(f"🚀 {len(points)} vectors pushed to Qdrant")

        time.sleep(1)

# ================= RUN =================
if __name__ == "__main__":
    try:
        resume_indexing_agent()  # ✅ no arguments needed
        print("\n🎯 Agent finished successfully")
    finally:
        mongo.close()
        print("✅ MongoDB closed")




🤖 AGENT STARTED: RESUME INDEXING
📊 MongoDB + AWS S3 + Qdrant
✅ AWS verified
✅ MongoDB connected
ℹ️ Qdrant collection exists


Loading weights: 100%|██████████| 103/103 [00:00<00:00, 335.24it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


✅ Embedding model loaded
📌 Pending/open resumes count: 15

📄 Processing 684d489a7d783615f6da851e | Job: 684d43cb7d783615f6da82c7
   📥 Downloading: uploads/1749895321436-Khushi singh resume  (1).pdf
❌ Failed → Unable to get page count. Is poppler installed and in PATH?

📄 Processing 684d4d617d783615f6da8706 | Job: 684d43cb7d783615f6da82c7
   📥 Downloading: uploads/1749896543506-WhatsApp Image 2025-06-14 at 3.45.00 PM.pdf
❌ Failed → Unable to get page count. Is poppler installed and in PATH?

📄 Processing 684d57808c77325205caf83b | Job: 684d448a7d783615f6da8313
   📥 Downloading: uploads/1749899135124-WhatsApp Image 2025-06-14 at 4.26.09 PM.pdf
❌ Failed → Unable to get page count. Is poppler installed and in PATH?

📄 Processing 6852adcdbcf10b931eb7c420 | Job: 6852ad6bbcf10b931eb7c3af
   📥 Downloading: uploads/1750248908595-infosis.jfif
❌ Failed → Unable to get page count. Is poppler installed and in PATH?

📄 Processing 6868b4048a09e3fb00a486db | Job: 6868b03a8a09e3fb00a486b2
   📥 Download

In [18]:
import os
from dotenv import load_dotenv
from pymongo import MongoClient
from bson import ObjectId
from qdrant_client import QdrantClient
from sentence_transformers import SentenceTransformer
from bs4 import BeautifulSoup
import re

# ================= LOAD ENV =================
load_dotenv()

MONGO_URI = os.getenv("MONGODB_URI")
DB_NAME = "ats"

QDRANT_URL = os.getenv("QDRANT_URL")
QDRANT_API_KEY = os.getenv("QDRANT_API_KEY")
QDRANT_COLLECTION = "resumes"

print("\n🚀 JD MATCHING AGENT STARTED (Best Score Relative Mode)")

# ================= MongoDB =================
mongo = MongoClient(MONGO_URI)
db = mongo[DB_NAME]
companies = db["companies"]
applications = db["applications"]
jobs = db["jobs"]
job_statuses = db["job-statuses"]

print("✅ MongoDB connected")

# ================= QDRANT =================
qdrant = QdrantClient(
    url=QDRANT_URL,
    api_key=QDRANT_API_KEY,
    prefer_grpc=False
)
print("✅ Qdrant connected")

# ================= MODEL =================
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
print("✅ Embedding model loaded")

# ================= CLEAN HTML =================
def clean_html(raw_html):
    soup = BeautifulSoup(raw_html, "html.parser")
    text = soup.get_text(separator=" ")
    text = re.sub(r"\s+", " ", text).strip()
    return text

# ================= SEMANTIC JD MATCHING =================
def jd_matching_agent(company):
    company_id = company["_id"]
    print(f"\n🏢 Running JD Matching For Company: {company.get('name')}")
    print(f"🔎 Processing company: {company_id}")

    # 1️⃣ Get OPEN status
    open_status_doc = list(job_statuses.find({
        "company_id": ObjectId(company_id),
        "jobStatus": "Open"
    }))

    if not open_status_doc:
        print(f"❌ No OPEN status found for company {company.get('name')}")
        return

    open_status_id = open_status_doc[0]["_id"]

    # 2️⃣ Get OPEN jobs
    open_jobs = list(jobs.find({
        "company_id": ObjectId(company_id),
        "status": str(open_status_id)
    }))

    if not open_jobs:
        print("❌ No OPEN jobs found")
        return

    # 3️⃣ Process each open job
    for job in open_jobs:
        job_id = str(job["_id"])
        job_description = job.get("description", "")

        if not job_description.strip():
            print(f"⚠️ Job {job_id} has no description")
            continue

        print(f"\n🔎 Processing Job: {job_id}")

        cleaned_description = clean_html(job_description)
        jd_vector = model.encode(cleaned_description).tolist()

        try:
            # Fetch all resumes and filter in Python
            search_results = qdrant.query_points(
                collection_name=QDRANT_COLLECTION,
                query=jd_vector,
                limit=100000,  # Fetch all
                with_payload=True,
                with_vectors=False
            ).points

            # Filter only resumes for this job
            search_results = [
                r for r in search_results
                if r.payload.get("job_id") == job_id
            ]

            print(f"📊 Found {len(search_results)} resumes for this job")

            if not search_results:
                continue

            scores = [r.score for r in search_results if r.score is not None]

            if not scores:
                print("❌ No similarity scores found")
                continue

            # 🎯 Best-score based cutoff
            best_score = max(scores)
            cutoff = best_score * 0.63

            print(f"🏆 Best Score: {best_score:.4f}")
            print(f"🎯 Selection Cutoff (63% of Best): {cutoff:.4f}")

            # 4️⃣ Update application status
            for result in search_results:
                payload = result.payload or {}
                application_id = payload.get("application_id")
                score = result.score

                if not application_id or score is None:
                    continue

                status = "selected" if score >= cutoff else "rejected"

                applications.update_one(
                    {"_id": ObjectId(application_id)},
                    {"$set": {"resume_status": status}}
                )

                print(f"   ➜ {application_id} | {score:.4f} → {status}")

        except Exception as e:
            print(f"❌ Qdrant query failed: {e}")

    print("✅ Company JD Matching Done")

# ================= RUN =================
if __name__ == "__main__":
    try:
        ai_companies = list(companies.find({"aiFeaturesEnabled": True}))

        if not ai_companies:
            print("❌ No AI enabled companies found")
        else:
            print(f"🏢 Found {len(ai_companies)} AI enabled companies")

            for company in ai_companies:
                jd_matching_agent(company)

        print("\n🎯 JD Matching Completed Successfully")
    finally:
        mongo.close()
        print("✅ MongoDB closed")



🚀 JD MATCHING AGENT STARTED (Best Score Relative Mode)
✅ MongoDB connected
✅ Qdrant connected


Loading weights: 100%|██████████| 103/103 [00:00<00:00, 153.95it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


✅ Embedding model loaded
🏢 Found 3 AI enabled companies

🏢 Running JD Matching For Company: f2fintech
🔎 Processing company: 682858bb96c2ed0759146648

🔎 Processing Job: 684920beae8bd1780f520b09
📊 Found 0 resumes for this job

🔎 Processing Job: 684d43cb7d783615f6da82c7
📊 Found 161 resumes for this job
🏆 Best Score: 0.5407
🎯 Selection Cutoff (63% of Best): 0.3406
   ➜ 684d489c7d783615f6da8521 | 0.5407 → selected
   ➜ 68e76cac72a5ec1432f67f96 | 0.4242 → selected
   ➜ 68662f868e27a08a98339a69 | 0.4138 → selected
   ➜ 6958d29212dc988d45a83653 | 0.4052 → selected
   ➜ 690c3c73450fd808275d0ef3 | 0.3896 → selected
   ➜ 68a990b40f7a9b9b9c5c9059 | 0.3778 → selected
   ➜ 69391342483e6754327e9e82 | 0.3711 → selected
   ➜ 68d3be5380642c2dbf4d5b64 | 0.3679 → selected
   ➜ 68b68135dbdbc7d4e8742ed6 | 0.3607 → selected
   ➜ 69391281483e6754327e9e4b | 0.3597 → selected
   ➜ 693914c7483e6754327e9ee9 | 0.3512 → selected
   ➜ 68b9273a97d1801b76070552 | 0.3490 → selected
   ➜ 68904ab933d56038c0547863 | 0.347